<a href="https://colab.research.google.com/github/JacopoBartoli/vas_regression/blob/main/Transformer_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1) Install packages and organize imports.
In this section we install the needed packages and import them.
We set some variables for the used paths, and mount GDrive.

In [ ]:
!pip install tensorflow-addons

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import tensorboard
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import datetime
import math

In [ ]:
DATASET_DIR = '/content/gdrive/My Drive/IVA/data/'
LOGS_DIR = '/content/gdrive/My Drive/IVA/logs'
CHECKPOINT_DIR = '/content/gdrive/My Drive/IVA/checkpoint/train'
MODEL_DIR = '/content/gdrive/My Drive/IVA/model'

Mount the drive.

In [ ]:
# Mount your drive to access the dataset.
# Remember to link the dataset as explained above.
from google.colab import drive
drive.mount('/content/gdrive')
!ls -l "/content/gdrive/My Drive/"
#!rm -rf './IVA/logs/gradient_tape/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ls: '/content/gdrive/My Drive/DeepFashion2 Dataset': No such file or directory
total 2794
drwx------ 2 root root   4096 Mar 18 14:03  3D_result
-rw------- 1 root root 555998 Dec  3  2020  7036362.pdf
-rw------- 1 root root   6420 Dec  2  2019 '7036362-sol (1).pdf'
-rw------- 1 root root   6669 Dec  3  2020  7036362-sol.pdf
-rw------- 1 root root 281590 Dec  8  2020  bartoli_jacopo_report2.pdf
drwx------ 2 root root   4096 Nov 28  2020 'Colab Notebooks'
drwx------ 2 root root   4096 Dec  1  2020  DDM
lrw------- 1 root root      0 Mar 24 15:34 'DeepFashion2 Dataset' -> '/content/gdrive/.shortcut-targets-by-id/125F48fsMBz2EF0Cpqk6aaHet5VH399Ok/DeepFashion2 Dataset'
drwx------ 6 root root   4096 Sep  3 11:19  IVA
-rw------- 1 root root 109584 Nov 10  2020  Jacopo_Bartoli_report.pdf
lrw------- 1 root root     25 Nov 30  2020 'My Drive' -> '/content/gdrive/My Dri

#2) Define our transformer.
In this section we implement our transformer model.

##2.1) Utility functions.
Define some utilities functions, for the positional encodings and the feed forward network.

In [ ]:
# Define the positional encoding function.
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates


def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
# Define the feed forward network
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
                              tf.keras.layers.Dense(dff, activation='relu'),
                              tf.keras.layers.Dense(d_model)
  ])

##2.2) Define the encoder layer.


In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = tf.keras.layers.MultiHeadAttention(num_heads, output_shape=d_model, key_dim=24)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):
    attn_output = self.mha(x,x,x,mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2

##2.3) Define the encoder.

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, rate=0.1):
    super(Encoder, self).__init__()
    
    self.d_model = d_model
    self.num_layers = num_layers

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate ) for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    return x

## 2.4) Define the transformer with the regression layer.

In [ ]:
class EncoderRegressor(tf.keras.Model):
  def __init__(self, feat_dim, max_len, d_model, n_heads, num_layers, dim_feedforward, num_classes=1, dropout=0.1, pos_encoding='fixed', activation='gelu', norm='BatchNorm'):
    super(EncoderRegressor, self).__init__()

    self.max_len = max_len
    self.d_model = d_model
    self.n_heads = n_heads
    
    self.flatten_inp = tf.keras.layers.Flatten()

    self.project_inp = tf.keras.layers.Dense(max_len*d_model)

    self.reshape = tf.keras.layers.Reshape((max_len,d_model))

    self.pos_encoding = positional_encoding(2048, self.d_model)


    self.encoder_layer = Encoder(num_layers = num_layers, d_model = d_model, num_heads = n_heads, dff= dim_feedforward, rate=0.01)

    self.act = tf.keras.activations.get(activation)
    self.dropout = tf.keras.layers.Dropout(dropout)

    self.flatten = tf.keras.layers.Flatten()
    
    self.reg = tf.keras.layers.Dense(num_classes)

    self.feat_dim = feat_dim
    self.num_classes = num_classes

  def call(self, inputs, training):    
      enc_padding_mask = None #create_padding_mask(inputs)
      seq_len = tf.shape(inputs)[1]

      # Flatten the input tensor and map in a different vector space(d_model)
      x = self.flatten_inp(inputs)
      x = self.project_inp(x)
      # Reshape the tensor to adapt the shape [batch_size, sequence_lenght, d_model]
      x = self.reshape(x)

      # Positional encoding.
      x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
      x += self.pos_encoding[:, :seq_len, :]
      
      # Encoder Layer
      x = self.encoder_layer(x, training, enc_padding_mask)
      x = self.dropout(x, training=training)

      # Regression Layers
      x = self.flatten(x)
      x = self.reg(x)

      return x


  

#3) Manage the data.
In this section we manipulate and extract the data.

##3.1) Load the test set.
Define the name of the dataset used for testing.



In [ ]:
# Name of the dataset used.
TEST_SET = 'test.csv'

Load the test set from a .csv file.

In [ ]:
df = pd.read_csv(DATASET_DIR + TEST_SET)
print(df.head())

   Sequenza  Frame      Vel1      Vel2  ...     Vel10     Vel11     Vel12  Label
0       180      0  0.085440  0.080623  ...  0.094340  0.050000  0.098995      0
1       180      1  0.098489  0.120000  ...  0.036056  0.067082  0.082462      0
2       180      2  0.200000  0.170000  ...  0.277308  0.245153  0.053852      0
3       180      3  0.078102  0.092195  ...  0.092195  0.120416  0.050000      0
4       180      4  0.182483  0.194165  ...  0.053852  0.086023  0.067082      0

[5 rows x 15 columns]


## 3.2) Extract the labels from the dataset.
Extract the test set column that contains the labels and group them by sequence id. In this way the output is a list of a single label for each sequence.

In [ ]:
# Extract the column that contains the test labels.
lbl = df['Label']
seq_ids = df['Sequenza']

temp = pd.concat([seq_ids, lbl], axis=1)
temp = temp.set_index('Sequenza')
temp = temp.groupby(level='Sequenza').mean()

lbl_test = temp['Label'].values

Remove the label column from the dataframe, and transform the data in the correct input format. The sequences need to be numbered from 0 to $num\_seq - 1$.

In [ ]:
# Drop the label column.
data = df.drop(['Label'], axis = 1)
min_seq = data['Sequenza'].min()
num_seqs = data['Sequenza'].max() - data['Sequenza'].min() + 1

# Create the new dataset.
temp = []
for id in tqdm(range(min_seq, min_seq + num_seqs)):
  # Extract sequences one by one.
  seq = data.loc[data['Sequenza'] == id]

  # Remove the unused columns.
  seq = seq.drop(['Sequenza','Frame'], axis=1)
  num_col = len(seq.columns)

  # Iterate over each row of the selected sequence  
  temp_row = []
  for index, row in seq.iterrows():
    temp_row = np.append(temp_row, row)
  temp_row = np.reshape(temp_row, (-1, num_col))
  # add padding mas

  temp.append(temp_row[:])

data = temp

100%|██████████| 20/20 [00:00<00:00, 50.59it/s]


In [ ]:
data = tf.keras.preprocessing.sequence.pad_sequences(data, maxlen=681, dtype='float64')

In [ ]:
#print(len(data[100]))
#print(len(data[160]))
#print(data[100][675])
#print(data[160][641])

## 3.3) Create and manage the test set.
Transform the dataset ina tensorflow.data.Dataset format.

In [ ]:
ds = tf.data.Dataset.from_tensor_slices((data, lbl_test))

Define batch size and other variables.

In [ ]:
BATCH_SIZE = 1
BUFFER_SIZE = 180
random_seed = 1337

In [ ]:
def make_batches(ds):
  return (
      ds
      .cache()
      .shuffle(BUFFER_SIZE,seed=random_seed)
      .batch(BATCH_SIZE)
      .prefetch(tf.data.AUTOTUNE))

In [ ]:
test_batches = make_batches(ds)

#4) Evaluation.

## 4.1) Set the hyperparameters.

In [ ]:
# Model hyperparameters
d_model = 32
dim_feedforward = 256
n_heads = 6
num_layers = 3
feat_dim = 12 # Number of feature inside each item of the sequence.
max_len=  681 # Lenght of each sequence.

# Parameter needed for separate classification from regression.
# For now just regression is implemented.
num_classes = 1
is_classification = False


# Network hyperparameter
learning_rate = 0.001
optimizer = tf.keras.optimizers.Adam()
# tfa.optimizers.RectifiedAdam() seems to not work properly.

# This loss and accuracy objects are meant for regression.
# For classifications other metrics will be needed.
accuracy_object = tf.keras.metrics.MeanAbsoluteError()

## 4.2) Load the model.
Since it's a custom model it can't be load as .h5 config. We need to load the weight of the last epoch of the training.

In [ ]:
!ls -l '/content/gdrive/My Drive/IVA/model'

total 8
drwx------ 4 root root 4096 Sep 16 09:08 20210916-081824
drwx------ 4 root root 4096 Sep 16 09:03 20210916-081824_Transformers.h5


In [ ]:
# In this case we use the model associated to the train of the 2021-09-15 at 20:22:03
current_time = datetime.datetime(2021, 9, 16, 8, 18, 24).strftime("%Y%m%d-%H%M%S")
#transformer = tf.saved_model.load(MODEL_DIR + '/' + current_time + '/Transformer.h5/assets')
transformer = tf.saved_model.load('/content/gdrive/My Drive/IVA/model/20210916-081824/transformers')

In [ ]:
# Create the transformer.
transformer = EncoderRegressor(d_model=d_model, dim_feedforward=dim_feedforward, n_heads=n_heads, num_layers=num_layers, feat_dim=feat_dim, max_len=max_len, num_classes = num_classes)

# Checkpoint management.
ckpt = tf.train.Checkpoint(trasformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, CHECKPOINT_DIR, max_to_keep=5)
print(ckpt_manager.latest_checkpoint)
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print("Restored latest checkpoint")

## 4.2) Custom implementation of the accuracy function. 
Add a way to customize the accuracy function.

In [ ]:
def accuracy_function(real, pred):

  accuracies = accuracy_object(real, pred)
  
  return accuracies

Create the metrics object.

In [ ]:
test_accuracy = tf.keras.metrics.Mean(name='test_accuracy')

## 4.3) Set the paths for Tensorboard.

The test_log_dir need to be associate to a valid train_log_dir using their timestamp.

In [ ]:
# In this case we use the model associated to the train of the 2021-09-15 at 20:22:03
current_time = datetime.datetime(2021, 9, 16, 8, 18, 24).strftime("%Y%m%d-%H%M%S")
test_log_dir = LOGS_DIR + '/gradient_tape/' + current_time + '/test'
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

## 4.2) Evaluate the model.

In [ ]:
start = time.time()
  
test_accuracy.reset_states()

# Needed for histogram visualization.
predictions_histogram = []
labels_histogram = []


for (batch, (inp, tar)) in enumerate(test_batches):
    predictions = transformer(inp, training = False)
    
    accuracy = accuracy_function(tar, predictions)
    test_accuracy(accuracy)
    # Save the histogram of predictions.
    predictions_histogram = np.hstack((predictions_histogram, tf.reshape(predictions, len(predictions))))    
    labels_histogram = np.hstack((labels_histogram, tar))
with test_summary_writer.as_default():
   tf.summary.scalar('accuracy', test_accuracy.result(),step = 0)
   tf.summary.histogram('predictions distribution', predictions_histogram, step=0)
   tf.summary.histogram('label distribution', labels_histogram, step=0)

TypeError: ignored